In [1]:
import pandas as pd 
import numpy as np
import networkx as nx 
import json 
import csv 
import matplotlib as plt 
import cdlib
import os


In [3]:
g = nx.read_weighted_edgelist('../data/connectedJ.weighted.edgelist')

In [5]:



for edge in g.edges():
    g.edges[edge]['dissimilarity'] = 10001 - g.edges[edge]['weight']

    

In [6]:
def coreNode(g,alfa,weight):
    
    
    def localCloseNeighborThreshold(g,p,weight):     
        geomin = 1   
        for q in g.neighbors(p): geomin *= g.edges[(p,q)][weight]
        return geomin ** (1 / len(list(g.neighbors(p))))
    
    def localCloseNeighbors(g,p,tresholds,weight):
        return [ q for q in g.neighbors(p) if g.edges[(p,q)][weight] <= tresholds[p] and g.edges[(p,q)][weight] <= tresholds[q] ]
    
        
    
    localCloseNeighborThresholds = {p:localCloseNeighborThreshold(g,p,weight) for p in g.nodes()}
    
    corenodes = list()
    for node in g.nodes:
        
        localMinimumClusteringThreshold = alfa * len(list(g.neighbors(node)))
        treshold = localCloseNeighborThresholds[node]
        
        if len(localCloseNeighbors(g,node,localCloseNeighborThresholds,weight)) >= localMinimumClusteringThreshold: 
            
            corenodes.append(node)
        
    return corenodes

In [17]:
def multicore(G,alfa=0.3,weight='weight',minsize = 4, voteweight = 'centrality'):
    
    #PREPARATION
    g = G.copy()
    core = coreNode(g,alfa,weight)
    communities = list()
    centrality = 1
    k = 0
    borders = set()
    
    for node in g.nodes():
        g.nodes[node]['visited'] = False
        g.nodes[node]['community'] = None
        if node in core: g.nodes[node]['core']=True 
        else : g.nodes[node]['core']=False


            
    #INITIALIZE COMMUNITIES
    for p in g.nodes():

        if g.nodes[p]['visited'] == True or g.nodes[p]['core'] == False: continue
        
        candidates = set(g.neighbors(p))
        
        communities.append([p])
        g.nodes[p]['community'] = k
        g.nodes[p]['centrality'] = centrality
        g.nodes[p]['visited'] = True
        
        while len(candidates) != 0:

            q = candidates.pop()
           
            if g.nodes[q]['visited'] == False:

                if g.nodes[q]['core'] == True :   
                    communities[k].append(q)
                    g.nodes[q]['community'] = k
                    g.nodes[q]['centrality'] = centrality
                    for x in g.neighbors(q):
                        if g.nodes[x]['visited'] == False : candidates.add(x)
                
                else: borders.add(q)
                g.nodes[q]['visited'] = True

        k = k + 1
        
    
    #PROPAGATION
  
    while len(borders) != 0:
        newborders = set() 
        for p in borders: 
            votes = [0 for c in communities]
            for q in g.neighbors(p):
                if q not in borders:
                    if g.nodes[q]['community'] != None:
                        if voteweight == 'centrality': votes[g.nodes[q]['community']] += 1 / g.nodes[q]['centrality']
                        elif voteweight == 'weight': votes[g.nodes[q]['community']] += 1 / g.edges[(p,q)][weight]
                    else: newborders.add(q)
            k = votes.index(max(votes))
            communities[k].append(p)
            g.nodes[p]['community'] = k
            g.nodes[p]['centrality'] = centrality
            
        borders = newborders
        centrality += 1
        

 
       #OUTLIERS 
    unmarked = list()
    for k,c  in enumerate(communities): 
        if len(c) <= minsize :
            
            for p in c: 
                g.nodes[p]['community'] = None
                
            communities[k] = []    
            unmarked.extend(c)
            
    borders = set()
    for p in unmarked:
        for q in g.neighbors(p):
            if g.nodes[q]['community'] != None:
                borders.add(p)
                break
                
    borders = set(borders)
    
   
    while len(borders) != 0:
        newborders = set() 
        for p in borders: 
            votes = [0 for c in communities]
            for q in g.neighbors(p):
                if q not in borders:
                    if g.nodes[q]['community'] != None:
                        if voteweight == 'centrality': votes[g.nodes[q]['community']] += 1 / g.nodes[q]['centrality']
                        elif voteweight == 'weight': votes[g.nodes[q]['community']] += 1 / g.edges[(p,q)][weight]
                    else: newborders.add(q)
            k = votes.index(max(votes))
            communities[k].append(p)
            g.nodes[p]['community'] = k
            g.nodes[p]['centrality'] = 1 + min([g.nodes[q]['centrality'] for q in g.neighbors(p) if g.nodes[q]['community'] == k ])
            
        borders = newborders
        
            

    communities = [c for c in communities if len(c) != 0 ]
    data = dict()
    data['communities'] = communities
    data['outliers'] = list(node for node in g.nodes() if g.nodes[node]['community'] == None)
    data['coverage']= 1
    data['algorithm'] = 'multicore'
    data['centralities'] = {node:g.nodes[node]['centrality'] for node in g.nodes}
        
    return data
                        
                    
                    
                
            
            

In [28]:
def multicore(G,alfa=0.7,weight='weight',minsize = 4):
    
    #PREPARATION
    g = G.copy()
    core = coreNode(g,alfa,weight)
    communities = list()
    centrality = 1
    k = 0
    borders = set()
    
    for node in g.nodes():
        g.nodes[node]['visited'] = False
        g.nodes[node]['community'] = None
        if node in core: g.nodes[node]['core']=True 
        else : g.nodes[node]['core']=False


            
    #INITIALIZE COMMUNITIES
    for p in g.nodes():

        if g.nodes[p]['visited'] == True or g.nodes[p]['core'] == False: continue
        
        candidates = set(g.neighbors(p))
        
        communities.append([p])
        g.nodes[p]['community'] = k
        g.nodes[p]['centrality'] = centrality
        g.nodes[p]['visited'] = True
        
        while len(candidates) != 0:

            q = candidates.pop()
           
            if g.nodes[q]['visited'] == False:

                if g.nodes[q]['core'] == True :   
                    communities[k].append(q)
                    g.nodes[q]['community'] = k
                    g.nodes[q]['centrality'] = centrality
                    for x in g.neighbors(q):
                        if g.nodes[x]['visited'] == False : candidates.add(x)
                
                else: borders.add(q)
                g.nodes[q]['visited'] = True

        k = k + 1
        
    
    #PROPAGATION
  
    while len(borders) != 0:
        newborders = set() 
        for p in borders: 
            centrality = None
            votes = [0 for c in communities]
            for q in g.neighbors(p):
                
                if q not in borders:                        
                    if g.nodes[q]['community'] != None:
                        voteweight = g.nodes[q]['centrality'] + g.edges[(p,q)][weight]
                        if centrality == None or voteweight < centrality:
                            centrality = voteweight
                        votes[g.nodes[q]['community']] += 1 / voteweight
                    else: newborders.add(q)
            k = votes.index(max(votes))
            communities[k].append(p)
            g.nodes[p]['community'] = k
            
            g.nodes[p]['centrality'] = centrality
            
        borders = newborders
        centrality += 1
        

 
       #OUTLIERS 
    unmarked = list()
    for k,c  in enumerate(communities): 
        if len(c) <= minsize :
            
            for p in c: 
                g.nodes[p]['community'] = None
                
            communities[k] = []    
            unmarked.extend(c)
            
    borders = set()
    for p in unmarked:
        for q in g.neighbors(p):
            if g.nodes[q]['community'] != None:
                borders.add(p)
                break
                
    borders = set(borders)
    
   
    while len(borders) != 0:
        newborders = set() 
        for p in borders: 
            votes = [0 for c in communities]
            centrality = None
            for q in g.neighbors(p):
                if q not in borders:
                    if g.nodes[q]['community'] != None:
                        voteweight = g.nodes[q]['centrality'] + g.edges[(p,q)][weight]
                        if centrality == None or voteweight < centrality:
                            centrality = voteweight
                        votes[g.nodes[q]['community']] += 1 / voteweight
                    else: newborders.add(q)
            k = votes.index(max(votes))
            communities[k].append(p)
            g.nodes[p]['community'] = k
            g.nodes[p]['centrality'] = centrality
            
        borders = newborders
        
            

    communities = [c for c in communities if len(c) != 0 ]
    data = dict()
    data['communities'] = communities
    data['outliers'] = list(node for node in g.nodes() if g.nodes[node]['community'] == None)
    data['coverage']= 1
    data['algorithm'] = 'multicore'
    data['centralities'] = {node:g.nodes[node]['centrality'] for node in g.nodes}
        
    return data
                        
                    
                    
                
            
            

In [29]:
print(len(g.nodes))
print(len(coreNode(g,0.,'dissimilarity')))

15485
141


In [36]:
comm = multicore(g,0.7,'dissimilarity',minsize = 300)

In [37]:
print(len(comm['communities']))
print([len(c) for c in comm['communities']])

15
[1034, 737, 1004, 915, 1375, 731, 1822, 767, 1203, 639, 982, 1043, 1030, 1269, 934]
